<a href="https://colab.research.google.com/github/SahibJ56/Fruit-and-Vegetables-Classifier/blob/main/fruit_and_veggies_(image_classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Harika had made and trained deep learning model capable of classifying fruits and vegetables into their respective categories, after this now Once the model's performance is evaluated, Harika can use the trained model to classify new images of fruits and vegetables in real-time. Let’s help her now to deploy this model into an automated system at the farm for continuous use.



TASK:

- Load the Trained Model: The trained deep learning model needs to be loaded from the file .
- Preprocess the New Image: The new image should be resized, normalized, and transformed (if necessary) to match the format used for training.
- Make Predictions: The image can then be passed through the model to get the predicted class label.
- Display the Prediction: Harika can display the class label with the highest predicted probability as the final result.






In [ ]:
!pip install tensorflow opencv-python

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os
import matplotlib.pyplot as plt


In [ ]:
dataset_path = '/content/fruits-360/Training' #change path when testing

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
def create_model(input_shape=(224, 224, 3), num_classes=5):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))

    return model


In [ ]:
model = create_model(input_shape=(224, 224, 3), num_classes=len(train_generator.class_indices))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(train_generator, epochs=5, validation_data=val_generator)


In [ ]:
model.save('/content/fruit_classifier_model.h5')
print("Model saved!")


In [ ]:
loaded_model = load_model('/content/fruit_classifier_model.h5')
print("Model reloaded!")


In [ ]:
new_image_path = '/content/test_image.jpg' #change path when testing


def preprocess_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Could not read image from {img_path}")

    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)

    return img


def predict_class(model, img, class_labels):
    predictions = model.predict(img)
    predicted_class_index = np.argmax(predictions)
    predicted_probability = np.max(predictions)
    predicted_class = class_labels[predicted_class_index]

    return predicted_class, predicted_probability


class_labels = list(train_generator.class_indices.keys())
print("Class labels:", class_labels)


preprocessed_image = preprocess_image(new_image_path)


predicted_class, predicted_probability = predict_class(loaded_model, preprocessed_image, class_labels)


print(f"Predicted Class: {predicted_class}")
print(f"Confidence: {predicted_probability:.2f}")


img_show = cv2.imread(new_image_path)
img_show = cv2.cvtColor(img_show, cv2.COLOR_BGR2RGB)
plt.imshow(img_show)
plt.title(f"Prediction: {predicted_class} ({predicted_probability:.2f})")
plt.axis('off')
plt.show()
